In [22]:
cd /Users/owenje/Documents/339_2023_reboot/labs/reads/
ls

genome_10_reads1.fq		genome_5_reads2.fq
genome_10_reads2.fq		genome_6_reads1.fq
genome_1_reads1.fq		genome_6_reads2.fq
genome_1_reads2.fq		genome_7_reads1.fq
genome_2_reads1.fq		genome_7_reads2.fq
genome_2_reads2.fq		genome_8_reads1.fq
genome_3_reads1.fq		genome_8_reads2.fq
genome_3_reads2.fq		genome_9_no_cb_reads1.fq
genome_4_reads1.fq		genome_9_no_cb_reads2.fq
genome_4_reads2.fq		genome_9_plus_cb_reads1.fq
genome_5_reads1.fq		genome_9_plus_cb_reads2.fq


In [23]:
ls *reads1*

genome_10_reads1.fq	genome_4_reads1.fq	genome_8_reads1.fq
genome_1_reads1.fq	genome_5_reads1.fq	genome_9_reads1.fq
genome_2_reads1.fq	genome_6_reads1.fq
genome_3_reads1.fq	genome_7_reads1.fq


In [24]:
fw_reads=(*reads1*)

In [25]:
echo ${fw_reads[0]}
echo ${fw_reads[1]}
echo ${fw_reads[2]}
echo ${fw_reads[9]}

genome_10_reads1.fq
genome_1_reads1.fq
genome_2_reads1.fq
genome_9_reads1.fq


In [26]:
echo ${fw_reads[11]}

In [27]:
for i in ${fw_reads[@]}
do
    echo $i
done

genome_10_reads1.fq
genome_1_reads1.fq
genome_2_reads1.fq
genome_3_reads1.fq
genome_4_reads1.fq
genome_5_reads1.fq
genome_6_reads1.fq
genome_7_reads1.fq
genome_8_reads1.fq
genome_9_reads1.fq


In [28]:
for i in ${!fw_reads[@]}
do
    echo $i
done

0
1
2
3
4
5
6
7
8
9


In [29]:
for i in ${!fw_reads[@]}
do
    echo ${fw_reads[$i]}
done

genome_10_reads1.fq
genome_1_reads1.fq
genome_2_reads1.fq
genome_3_reads1.fq
genome_4_reads1.fq
genome_5_reads1.fq
genome_6_reads1.fq
genome_7_reads1.fq
genome_8_reads1.fq
genome_9_reads1.fq


#### Now have students do the same for reverse reads. Don't provide answer below

hint

```rv_reads=(?..what to put here..?)```

In [30]:
rv_reads=(*reads2*)

In [31]:
for i in ${!fw_reads[@]}
do
    echo ${fw_reads[$i]} "+" ${rv_reads[$i]}
done

genome_10_reads1.fq + genome_10_reads2.fq
genome_1_reads1.fq + genome_1_reads2.fq
genome_2_reads1.fq + genome_2_reads2.fq
genome_3_reads1.fq + genome_3_reads2.fq
genome_4_reads1.fq + genome_4_reads2.fq
genome_5_reads1.fq + genome_5_reads2.fq
genome_6_reads1.fq + genome_6_reads2.fq
genome_7_reads1.fq + genome_7_reads2.fq
genome_8_reads1.fq + genome_8_reads2.fq
genome_9_reads1.fq + genome_9_reads2.fq


In [32]:
for i in ${!fw_reads[@]}
do
    reads_1=${fw_reads[$i]} 
    reads_2=${rv_reads[$i]}
    genome_name=${reads_1%??????????}  #remove last n characters, wher number of ?s = n

    AdapterRemoval --file1 $reads_1 --file2 $reads_2 --basename $genome_name \
    --trimns --trimqualities --collapse
done

Trimming paired end reads ...
Opening FASTQ file 'genome_10_reads1.fq', line numbers start at 1
Opening FASTQ file 'genome_10_reads2.fq', line numbers start at 1
Processed a total of 23,636 reads in 0.2s; 114,000 reads per second on average ...
Trimming paired end reads ...
Opening FASTQ file 'genome_1_reads1.fq', line numbers start at 1
Opening FASTQ file 'genome_1_reads2.fq', line numbers start at 1
Processed a total of 21,800 reads in 0.3s; 76,000 reads per second on average ...
Trimming paired end reads ...
Opening FASTQ file 'genome_2_reads1.fq', line numbers start at 1
Opening FASTQ file 'genome_2_reads2.fq', line numbers start at 1
Processed a total of 23,550 reads in 0.4s; 53,000 reads per second on average ...
Trimming paired end reads ...
Opening FASTQ file 'genome_3_reads1.fq', line numbers start at 1
Opening FASTQ file 'genome_3_reads2.fq', line numbers start at 1
Processed a total of 23,650 reads in 0.3s; 70,000 reads per second on average ...
Trimming paired end reads ...

In [33]:
mv *pair* ../trimmed_reads

In [34]:
#need to set this for negative pattern matching to work.
shopt -s extglob 

In [35]:
#ls !(*.fq)

In [36]:
rm !(*.fq)

In [37]:
cd ../trimmed_reads

In [38]:
#add .fq extension or spades won't accept the files
for f in *.truncated; do mv "$f" "$f.fq"; done

In [ ]:
fw_reads=(*pair1*)
rv_reads=(*pair2*)
for i in ${!fw_reads[@]}
do
    reads_1=${fw_reads[$i]} 
    reads_2=${rv_reads[$i]}
    mkdir $i
    echo $reads_1 $reads_2
    spades.py --pe1-1 $reads_1 --pe1-2 $reads_2 \
    -o $i 
done

genome_1.pair1.truncated.fq genome_1.pair2.truncated.fq


== Warning ==  No assembly mode was specified! If you intend to assemble high-coverage multi-cell/isolate data, use '--isolate' option.


Command line: /Users/owenje/anaconda3/envs/spades/bin/spades.py	--pe1-1	/Users/owenje/Dropbox/My Mac (fvfd52hvp3yv.staff.vuw.ac.nz)/Documents/339_2023_reboot/labs/trimmed_reads/genome_1.pair1.truncated.fq	--pe1-2	/Users/owenje/Dropbox/My Mac (fvfd52hvp3yv.staff.vuw.ac.nz)/Documents/339_2023_reboot/labs/trimmed_reads/genome_1.pair2.truncated.fq	-o	/Users/owenje/Dropbox/My Mac (fvfd52hvp3yv.staff.vuw.ac.nz)/Documents/339_2023_reboot/labs/trimmed_reads/0	

System information:
  SPAdes version: 3.15.2
  Python version: 3.9.18
  OS: macOS-12.3.1-x86_64-i386-64bit

Output dir: /Users/owenje/Dropbox/My Mac (fvfd52hvp3yv.staff.vuw.ac.nz)/Documents/339_2023_reboot/labs/trimmed_reads/0
Mode: read error correction and assembling
Debug mode is turned OFF

Dataset parameters:
  Standard mode
  For multi-ce

In [2]:
cd /Users/owenje/Documents/339_2023_reboot/labs/trimmed_reads
mkdir ../assemblies

In [3]:
fw_reads=(*pair1*)
for i in ${!fw_reads[@]}
do
    echo $i
    file_name=${fw_reads[$i]} 
    new_name="${file_name%%.*}.fna"
    echo $new_name
    contig_file=$i/contigs.fasta
    head -n 1 $contig_file
    cp $contig_file ../assemblies/$new_name
    
done

0
genome_1.fna
>NODE_1_length_65499_cov_24.041806
1
genome_10.fna
>NODE_1_length_69578_cov_24.027899
2
genome_2.fna
>NODE_1_length_70716_cov_23.833194
3
genome_3.fna
>NODE_1_length_70951_cov_23.952253
4
genome_4.fna
>NODE_1_length_71348_cov_23.934532
5
genome_5.fna
>NODE_1_length_71099_cov_24.003210
6
genome_6.fna
>NODE_1_length_71297_cov_23.971525
7
genome_7.fna
>NODE_1_length_71428_cov_23.952110
8
genome_8.fna
>NODE_1_length_71188_cov_23.939278
9
genome_9.fna
>NODE_1_length_74511_cov_23.967945


In [5]:
cd ../assemblies


In [6]:
ls

genome_1.fna	genome_2.fna	genome_4.fna	genome_6.fna	genome_8.fna
genome_10.fna	genome_3.fna	genome_5.fna	genome_7.fna	genome_9.fna


In [7]:
genomes=(*.fna)
for genome in ${genomes[@]}
do
    long-orfs $genome long_orfs.txt
    extract -t $genome long_orfs.txt > run1.train
    build-icm -r run1.icm < run1.train
    glimmer3 $genome run1.icm run1
    extract -t $genome run1.predict > genes_$genome
    rm run1.icm long_orfs.txt run1.train
done

Starting at Tue Sep 26 16:18:14 2023

Sequence file = genome_1.fna
Excluded regions file = none
Circular genome = true
Initial minimum gene length = 90 bp
Determine optimal min gene length to maximize number of genes
Maximum overlap bases = 30
Start codons = atg,gtg,ttg
Stop codons = taa,tag,tga
Sequence length = 65499
Final minimum gene length = 553
Number of genes = 33
Total bases = 53703
ERROR:  Skipped following coord line
Sequence file = genome_1.fna
ERROR:  Skipped following coord line
Excluded regions file = none
ERROR:  Skipped following coord line
Circular genome = true
ERROR:  Skipped following coord line
Initial minimum gene length = 90 bp
ERROR:  Skipped following coord line
Determine optimal min gene length to maximize number of genes
ERROR:  Skipped following coord line
Maximum overlap bases = 30
ERROR:  Skipped following coord line
Start codons = atg,gtg,ttg
ERROR:  Skipped following coord line
Stop codons = taa,tag,tga
ERROR:  Skipped following coord line
Sequence lengt

In [8]:
#add file name to orf headers, no idea how this works, but it does.
#makes a copy of the original file with -e and the end
for f in genes_*.fna; do sed -i -e "s/^>/>${f%.fna}_/g" "${f}"; done

In [10]:
rm *fna-e

In [13]:
pwd

/Users/owenje/Documents/339_2023_reboot/labs/assemblies


https://github.com/griffithlab/rnaseq_tutorial_wiki/blob/master/Kallisto.md

In [14]:
cat genes* > all_transcripts.fna

In [15]:
ls

all_transcripts.fna	genes_genome_7.fna	genome_5.fna
genes_genome_1.fna	genes_genome_8.fna	genome_6.fna
genes_genome_10.fna	genes_genome_9.fna	genome_7.fna
genes_genome_2.fna	genome_1.fna		genome_8.fna
genes_genome_3.fna	genome_10.fna		genome_9.fna
genes_genome_4.fna	genome_2.fna		run1.detail
genes_genome_5.fna	genome_3.fna		run1.predict
genes_genome_6.fna	genome_4.fna


In [16]:
kallisto index -i kallisto_index all_transcripts.fna --make-unique


[build] loading fasta file all_transcripts.fna
[build] k-mer length: 31
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Finished
CompactedDBG::build(): Estimated number of k-mers occurring at least once: 599653
CompactedDBG::build(): Estimated number of minimizer occurring at least once: 149899
CompactedDBG::filter(): Processed 600219 k-mers in 651 reads
CompactedDBG::filter(): Found 598482 unique k-mers
CompactedDBG::filter(): Number of blocks in Bloom filter is 4101
CompactedDBG::construct(): Extract approximate unitigs (1/2)
CompactedDBG::construct(): Extract approximate unitigs (2/2)
CompactedDBG::construct(): Closed all input files

CompactedDBG::construct(): Splitting unitigs (1/2)

CompactedDBG::construct(): Splitting unitigs (2/2)
CompactedDBG::construct(): Before split: 933 unitigs
CompactedDBG::construct(): After split (1/1): 933 unitigs
Compa

In [17]:
pwd

/Users/owenje/Documents/339_2023_reboot/labs/assemblies


In [18]:
cd ../patient_data

In [19]:
fw_reads=(*_1.fq)
rv_reads=(*_1.fq)

In [20]:
ls

patient_10_1.fq	patient_12_2.fq	patient_3_1.fq	patient_5_2.fq	patient_8_1.fq
patient_10_2.fq	patient_1_1.fq	patient_3_2.fq	patient_6_1.fq	patient_8_2.fq
patient_11_1.fq	patient_1_2.fq	patient_4_1.fq	patient_6_2.fq	patient_9_1.fq
patient_11_2.fq	patient_2_1.fq	patient_4_2.fq	patient_7_1.fq	patient_9_2.fq
patient_12_1.fq	patient_2_2.fq	patient_5_1.fq	patient_7_2.fq


In [21]:
mkdir ../outputs

In [22]:
for i in ${!fw_reads[@]}
do
   
   fw_file=${fw_reads[$i]}
   rv_file=${fw_reads[$i]}
   out_dir=../outputs/$fw_file
   mkdir $out_dir
   kallisto quant -i ../assemblies/kallisto_index -o $out_dir $fw_file $rv_file
done


[quant] fragment length distribution will be estimated from the data
[index] k-mer length: 31
[index] number of targets: 651
[index] number of k-mers: 599,040
[quant] running in paired-end mode
[quant] will process pair 1: patient_10_1.fq
                             patient_10_1.fq
[quant] finding pseudoalignments for the reads ... done
[quant] processed 19,195 reads, 19,191 reads pseudoaligned
[quant] estimated average fragment length: 0
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 52 rounds


[quant] fragment length distribution will be estimated from the data
[index] k-mer length: 31
[index] number of targets: 651
[index] number of k-mers: 599,040
[quant] running in paired-end mode
[quant] will process pair 1: patient_11_1.fq
                             patient_11_1.fq
[quant] finding pseudoalignments for the reads ... done
[quant] processed 18,985 reads, 18,978 reads pseudoaligned
[quant] estimated average fragment length: 0


In [23]:
ls

patient_10_1.fq	patient_12_2.fq	patient_3_1.fq	patient_5_2.fq	patient_8_1.fq
patient_10_2.fq	patient_1_1.fq	patient_3_2.fq	patient_6_1.fq	patient_8_2.fq
patient_11_1.fq	patient_1_2.fq	patient_4_1.fq	patient_6_2.fq	patient_9_1.fq
patient_11_2.fq	patient_2_1.fq	patient_4_2.fq	patient_7_1.fq	patient_9_2.fq
patient_12_1.fq	patient_2_2.fq	patient_5_1.fq	patient_7_2.fq


In [55]:
cd patient_data

In [56]:
ls *_1.fq

patient_10_1.fq	patient_1_1.fq	patient_4_1.fq	patient_7_1.fq
patient_11_1.fq	patient_2_1.fq	patient_5_1.fq	patient_8_1.fq
patient_12_1.fq	patient_3_1.fq	patient_6_1.fq	patient_9_1.fq


In [57]:
cp *_1.fq ../patient_data_single

In [60]:
cd ..

In [61]:
kallisto quant -i orf_files/kallisto_index -o outputs --single patient_data_single/*


Error: fragment length mean and sd must be supplied for single-end reads using -l and -s

Usage: kallisto quant [arguments] FASTQ-files

Required arguments:
-i, --index=STRING            Filename for the kallisto index to be used for
                              quantification
-o, --output-dir=STRING       Directory to write output to

Optional arguments:
-b, --bootstrap-samples=INT   Number of bootstrap samples (default: 0)
    --seed=INT                Seed for the bootstrap sampling (default: 42)
    --plaintext               Output plaintext instead of HDF5
    --single                  Quantify single-end reads
    --single-overhang         Include reads where unobserved rest of fragment is
                              predicted to lie outside a transcript
    --fr-stranded             Strand specific reads, first read forward
    --rf-stranded             Strand specific reads, first read reverse
-l, --fragment-length=DOUBLE  Estimated average fragment length
-s, --sd=DOUBLE  

: 1

In [62]:
cd ../patient_data

bash: cd: ../patient_data: No such file or directory


: 1

In [63]:
ls

orf_files		patient_data
outputs			patient_data_single


orf_files		patient_data
outputs			patient_data_single


In [77]:
kallisto quant -i orf_files/kallisto_index -o outputs patient_data/*


[quant] fragment length distribution will be estimated from the data
[index] k-mer length: 31
[index] number of targets: 655
[index] number of k-mers: 597,264
[quant] running in paired-end mode
[quant] will process pair 1: patient_data/patient_10_1.fq
                             patient_data/patient_10_2.fq
[quant] will process pair 2: patient_data/patient_11_1.fq
                             patient_data/patient_11_2.fq
[quant] will process pair 3: patient_data/patient_12_1.fq
                             patient_data/patient_12_2.fq
[quant] will process pair 4: patient_data/patient_1_1.fq
                             patient_data/patient_1_2.fq
[quant] will process pair 5: patient_data/patient_2_1.fq
                             patient_data/patient_2_2.fq
[quant] will process pair 6: patient_data/patient_3_1.fq
                             patient_data/patient_3_2.fq
[quant] will process pair 7: patient_data/patient_4_1.fq
                             patient_data/patient_4_2.fq
[

In [74]:
patient_data/*

bash: patient_data/patient_10_1.fq: Permission denied


: 126

In [75]:
ls

orf_files		patient_data
outputs			patient_data_single
